<a href="https://colab.research.google.com/github/charlesprice3/Roulette/blob/main/Roulette.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook acts as a simulation of a Roulette Wheel and aims to see if positive EV can be achieved through finding an edge.

TODO:

*   Implement Roulette Wheel Logic
*   Implement Roulette Memory Sliding Window
*   Implement mean reversion or ride the trend.

(Statistically speaking, each event is independently occurring, previous rolls bear no weight to future outcomes. This notebook tries to show this.)

*   Graph simulation of N playthorughs and see if we end up with positive EV



In [51]:
import random
import secrets
import numpy as np
import time
from IPython.display import clear_output  # This is the secret sauce for Notebooks

In [ ]:
memory = []
odd = 0
even = 0
n = 100000
for i in range(n):

  test = secrets.choice(range(0, 36))
  if(test % 2 == 0):
    even += 1
  else:
    odd += 1


print("Even: ", even)
print(n / 2 - even)

print("Odd: ", odd)
print(n /2 - odd)

Even:  49795
205.0
Odd:  50205
-205.0


In [54]:
def roll():
  return secrets.choice(range(0, 36))

In [59]:
memory = []
for i in range(10):
  memory.append(secrets.choice(range(0, 36)))
  k = 0

while k < 100:
  memory.pop(0)
  r = roll()
  memory.append(r)

  # wait=True prevents the "flicker" by waiting for new output before clearing the old
  clear_output(wait=True)

  # Now you can just print normally
  print(f"Roll: {r} ---- {memory}")

  # Adjust this speed.
  # 0.1 is usually the 'sweet spot' for Colab to keep up without lagging.
  time.sleep(.5)

  k += 1

Roll: 30 ---- [28, 2, 21, 3, 11, 32, 26, 29, 26, 30]


In [28]:
def playThrough(num):
  memory = []
  for i in range(10):
    memory.append(secrets.choice(range(0, 36)))
  for k in range(num):
    memory.pop(0)
    memory.append(secrets.choice(range(0, 36)))
  return memory


In [76]:
memory = playThrough(1000)
print(memory)

[6, 18, 28, 13, 2, 26, 26, 8, 34, 25]


In [ ]:
# Possible Routes
# Sum the memory box, take average, use as expected value for either specific value, or choose odd or even
# Check count of each odd/even occurrence, then either
  # "Ride the trend"
  # Use mean reversion concept

# More specifically, we can count longest values in a row that are odd or even from the back, and then make a sized bet based on how large this number is

In [77]:
even = 0
odd = 0
for i in range(len(memory)):
  if(memory[i] % 2 == 0):
    even += 1
  else:
    odd += 1

print("Even: ", even)
print("Odd: ", odd)

Even:  8
Odd:  2
